# FAI Coursework One - Game Search [15 Marks]
# Introduction


The main goal of coursework one is to give you first-hand experience on implementing search techniques for solving the problems in Python, one of the mostly adapted high level languages and analytic tools in business and research. The questions are designed to help on your understanding of concept of `minimax`, `alpha-beta prunning`, `adversrial problem formulation` and `heuristics` and their implementation. 

In completion of this project:

- Students are expected to master the knowledge of formulating adversrial game problem like `gobang` and `tic-tac-toe` game.
- Students are expected to use adversrial search algorithms to solve the `gobang` game problem.
- Students are expected know how to design and implement the **hearistics** to estimate the **untility value** of the **non-terminal** game board state of the `gobang` game, and evluating the posibility of a player winning the game.
- Students are able to use the hearistics to improve the efficiency of `adversrial search`.


## Requirements:


In this coursework, you are required to complete the implementation of `limited minimax search` and `limited alpha-beta search` methods for the `gobang` game, using a fixed depth of recursion and heuristic evaluation of non-terminal states. 

To finish this coursework, please study the python implementation of `tic-tac-toe game` problem and `alpha-beta search` algorithm in demo code `Game Search*.ipynb` and complete the implementation of `limited minimax search`, `limited alpha-beta search` methods and `goBang` class with the following **requirements**:

1. The `limited minimax search` and `limited alpha-beta search` will only considered `limited depth` of recursion (only considers a fixed number of moves ahead), instead of searching the game state until the terminal states.

2. The `limited minimax search` and `limited alpha-beta search` will make decision based **estimation** of the non-terminal state utility, rather than evaluating the **real** terminal state utility.

3. The algorithm must alternate between maximizing and minimizing the utility at each level of the tree, as in the original minimax and alpha-beta pruning algorithms.

3. There is a valid **heuristics** method implementation in `gobang` class to estimate **non-terminal** game board utility. 

        Heuristics are rules of thumb or shortcuts that are used to solve problems quickly and efficiently. They are often used when the optimal solution is too complex or too time-consuming to find. In the context of adversarial search, heuristics are used to estimate the utility of non-terminal game states. Heuristics can be based on domain-specific knowledge, such as the value of individual pieces in a chess game, or they can be based on general principles, such as the number of open lines in a game of Go. Heuristics are not guaranteed to provide an optimal solution, but they can be very effective in practice, especially when combined with other search techniques such as alpha-beta pruning.

4. The **Agent** player, using `limited minimax search` or `limited alpha-beta search`, can play the game intelligently. => When play the `gobang` game against a human player, **Agent** will not lose the game within **20 steps**.

5. The **Agent** player, using `limited minimax search` or `limited alpha-beta search`, can play the game efficiently. => When play the `gobang` game against a human player, **Agent** make every move within **20 seconds**.





# Marks

Coursework 1 accounts for 15% of the COMP1037 marks. Corresponding marks will be awarded for completing the code in `FAIcw1_2023.ipynb`. Both **Code** implementation and **report** (text explaination of your design) should be written using this file as the template.

1. Write your explaination using `Markdown` cell.
2. Use `Code` cell to input your code. 

**Note: a. please run your code in the notebook and upload your notebook together with all the outputs. b. avoid using problem setting requires more than 5 mins to get the output.**


# Plagiarism vs. Group Discussions

As you should know, there is no tolerance of plagiarism, and any breach of which will be dealt with according to the University's standard policies. Please be very careful not to cross the boundary into plagiarism while having general discussions regarding the coursework to promote the generation of new ideas and to enhance the learning experience. The important part is that when you sit down to actually do the work and write the answers, you do it individually. If you do this, and you truly understand what you have written, you will not be guilty of plagiarism. Do NOT, under any circumstances, share code or share figures, graphs or charts, etc.

# Deadline and submission procedure

The submission deadline is 5pm on the **12  April 2023** via Moodle. Late submission results into a 5% reduction of your coursework mark for each weekday. Any work handed in after the 29th April will receive zero marks.

Name your submission file: FAIcw1-XXX.ipynb, where XXX should be your student number, and submit a single file via Moodle.

If you can’t submit your coursework on time due to Extenuating Circumstances, please contact your personal tutor first. I am only granting an extension of submission based on his / her recommendation.

# Required libaray installation

Before start the coursework, please install the required library first.

step 1: Open conda prompt (windows user), or open terminal (macOS user)

step 2: In terminal/conda prompt, type : `pip install pygame`
(please ask for help, if you have problem with installing `pygame` )

In [ ]:
import math
import random
import time
import pygame
import sys
from collections import defaultdict


## Game board visualization

The following section of code will enable you interact the game board interface. You can use this code to print out the current state of the game board, as well as get input from players for their moves. You can tailor this code to your own game by modifying the `__init__` method, or just leave them unchanged. 

In [ ]:
class Visualize:
    def __init__(self):
        pygame.init()

        # define board size
        self.Height = 15 
        self.Width = 15
        # define colors
        self.BLACK = (41, 36, 33)
        self.WHITE = (255, 245, 238)
        self.BROWN = (174, 112, 0)
        self.BLUE = (68, 206, 246)
        self.YELLOW =(210, 180, 140)

        # define board and piece position and size
        self.WINDOW_SIZE = (700, 700)
        self.screen = pygame.display.set_mode(self.WINDOW_SIZE)
        self.pieces_surface = pygame.Surface(self.WINDOW_SIZE, pygame.SRCALPHA)
        self.highlight_surface = pygame.Surface(self.WINDOW_SIZE, pygame.SRCALPHA)

        # set window title
        pygame.display.set_caption("gobang")

        # fill background
        self.screen.fill(self.YELLOW)

        # define font
        self.font = pygame.font.Font(None, 40)

        # define board and piece position and size
        self.BOARD_POSITION = (50, 50)
        self.CELL_SIZE = 40
        self.BOARD_SIZE = (self.CELL_SIZE*self.Height, self.CELL_SIZE*self.Width)
        
 
    # draw board
    def draw_board(self):
        """Draw the board"""
        for i in range(self.Height+1):
            pygame.draw.line(self.screen, self.BLACK, (self.BOARD_POSITION[0], self.BOARD_POSITION[1] + i*self.CELL_SIZE), 
                (self.BOARD_POSITION[0]+self.BOARD_SIZE[0], self.BOARD_POSITION[1] + i*self.CELL_SIZE))
        for i in range(self.Width+1):
            pygame.draw.line(self.screen, self.BLACK, (self.BOARD_POSITION[0] + i*self.CELL_SIZE, self.BOARD_POSITION[1]), 
                (self.BOARD_POSITION[0] + i*self.CELL_SIZE, self.BOARD_POSITION[1]+self.BOARD_SIZE[1]))
        pygame.display.update()

    # draw piece
    def draw_piece(self, pos, color):
        """Draw a piece on the board, highlight the latest piece"""
        # print(pos)
        self.highlight_surface.fill(pygame.Color(0,0,0,0))
        pygame.draw.circle(self.pieces_surface, color, (self.BOARD_POSITION[0]+pos[0]*self.CELL_SIZE, self.BOARD_POSITION[1]+pos[1]*self.CELL_SIZE), 15)
        pygame.draw.circle(self.highlight_surface, self.BLUE, (self.BOARD_POSITION[0]+pos[0]*self.CELL_SIZE, self.BOARD_POSITION[1]+pos[1]*self.CELL_SIZE), 15, width=3)
        self.screen.blits([(self.pieces_surface, (0,0)), (self.highlight_surface, (0,0))])
        pygame.display.update()
        
    
    # draw end screen
    def draw_end_screen(self, winner):
        """Draw the end screen"""
        if winner == 1:
            text = self.font.render("Black wins!", True, self.BLACK)
        elif winner == 2:
            text = self.font.render("White wins!", True, self.BLACK)
        else:
            text = self.font.render("Draw!", True, self.BLACK)

        # Text background
        pygame.draw.rect(self.screen, self.WHITE, (self.WINDOW_SIZE[0]/2 - text.get_width()/2 - 10, self.WINDOW_SIZE[1]/3 - text.get_height()/2 - 10, text.get_width() + 20, text.get_height() + 20))
        pygame.draw.rect(self.screen, self.BLACK, (self.WINDOW_SIZE[0]/2 - text.get_width()/2 - 10, self.WINDOW_SIZE[1]/3 - text.get_height()/2 - 10, text.get_width() + 20, text.get_height() + 20), width=3)
        # Text
        self.screen.blit(text, (self.WINDOW_SIZE[0]/2 - text.get_width()/2, self.WINDOW_SIZE[1]/3 - text.get_height()/2))
        pygame.display.update()
        
        # Restart and Quit buttons
        pygame.draw.rect(self.screen, self.WHITE, (self.WINDOW_SIZE[0]/2 - 115, self.WINDOW_SIZE[1]/3 + 50, 110, 50))
        pygame.draw.rect(self.screen, self.BLACK, (self.WINDOW_SIZE[0]/2 - 115, self.WINDOW_SIZE[1]/3 + 50, 110, 50), width=3)
        pygame.draw.rect(self.screen, self.WHITE, (self.WINDOW_SIZE[0]/2 + 5, self.WINDOW_SIZE[1]/3 + 50, 95, 50))
        pygame.draw.rect(self.screen, self.BLACK, (self.WINDOW_SIZE[0]/2 + 5, self.WINDOW_SIZE[1]/3 + 50, 95, 50), width=3)     
        
        text = self.font.render("Restart", True, self.BLACK)
        self.screen.blit(text, (self.WINDOW_SIZE[0]/2 - 110 + 50 - text.get_width()/2, self.WINDOW_SIZE[1]/3 + 50 + 25 - text.get_height()/2))
        text = self.font.render("Quit", True, self.BLACK)
        self.screen.blit(text, (self.WINDOW_SIZE[0]/2 + 55 - text.get_width()/2, self.WINDOW_SIZE[1]/3 + 50 + 25 - text.get_height()/2))
        pygame.display.update()

        # Wait for user input
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()
                if event.type == pygame.MOUSEBUTTONUP:
                    mouse_x,mouse_y = pygame.mouse.get_pos()
                    if mouse_x > self.WINDOW_SIZE[0]/2 - 100 and mouse_x < self.WINDOW_SIZE[0]/2 and mouse_y > self.WINDOW_SIZE[1]/3 + 50 and mouse_y < self.WINDOW_SIZE[1]/3 + 50 + 50:
                        return 0 # restart
                    elif mouse_x > self.WINDOW_SIZE[0]/2 and mouse_x < self.WINDOW_SIZE[0]/2 + 100 and mouse_y > self.WINDOW_SIZE[1]/3 + 50 and mouse_y < self.WINDOW_SIZE[1]/3 + 50 + 50:
                        return 1 # quit

    # get mouse click position
    def get_input(self):
        """Get the mouse click position"""
        # wait for user input
        while True:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    sys.exit()   
                
                if event.type == pygame.MOUSEBUTTONUP:
                    mouse_x,mouse_y = pygame.mouse.get_pos()
                    # round to the nearest cell
                    x = round((mouse_x - self.BOARD_POSITION[0])/self.CELL_SIZE)
                    y = round((mouse_y - self.BOARD_POSITION[1])/self.CELL_SIZE)
                    # check if the click is on the board if not ignore
                    if x < self.BOARD_SIZE[0] and y < self.BOARD_SIZE[1]:
                        return x,y

## Game board implementation 

Similar to tic-tac-toe game, game states of `gobang` will be represented as a `Board`class object, which is a subclass of `defaultdict` that in general will consist of `{(x, y): contents}` pairs, for example `{(0, 0): 'X', (1, 1): 'O'}` might be the state of the board after two moves. Besides the contents of squares, a board also has some attributes: 
- `.to_move` to name the player whose move it is; 
- `.width` and `.height` to give the size of the board (both 3 in tic-tac-toe, but other numbers in related games);
- `.utility` to give utility value to the terminal state of the game, it is initially set as 0 by default;
- possibly other attributes, as specified by keywords. 

As a `defaultdict`, the `Board` class has a `__missing__` method, which returns `empty` for squares that have no been assigned but are within the `width` × `height` boundaries, or `off` otherwise. The class has a `__hash__` method, so instances can be stored in hash tables.

In [ ]:
class Board(defaultdict):
    """A board has the player to move, a cached utility value, 
    and a dict of {(x, y): player} entries, where player is 'X' or 'O'."""
    empty = '.'
    off = '#'
    
    def __init__(self, width=16, height=16, to_move=None, **kwds):
        self.__dict__.update(width=width, height=height, to_move=to_move, **kwds)
        
    def new(self, changes: dict, **kwds) -> 'Board':
        "Given a dict of {(x, y): contents} changes, return a new Board with the changes."
        board = Board(width=self.width, height=self.height, **kwds)
        board.update(self)
        board.update(changes)
        return board

    def __missing__(self, loc):
        x, y = loc
        if 0 <= x < self.width and 0 <= y < self.height:
            return self.empty
        else:
            return self.off
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)
    
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(map(row, range(self.height))) +  '\n'

## Limited-depth Game Search Algorithms ( Please complete the following code )


`Limited-depth` search is a search algorithm that limits the depth of the search tree. In adversarial games, this means that the algorithm only considers a fixed number of moves ahead, instead of searching all the way to the terminal states. This can be useful when the search space is too large to search exhaustively, or when the optimal solution is not required. 

The `limited minimax search` and `limited alpha-beta search` are adaptations of the minimax and alpha-beta pruning algorithms, respectively, that limit the depth of the search tree. 

### Requirement:

Implement `limited minimax search` and `limited alpha-beta search` methods for the `gobang` game, using a fixed depth `DEPTH_LIMIT` of recursion and heuristic evaluation of non-terminal states.


In [ ]:
def limited_minimax_search(game, state, DEPTH_LIMIT = 0):
    """Search game tree to determine best move; return (value, move) pair. DEPTH_LIMIT is the depth limit of the search tree."""
    
    
    player = state.to_move

    def max_value(state, DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state) :
             return game.utility(state, player), None
            
        v, move = -infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a, place):  
                if DEPTH_LIMIT == 0: 
                    new_state = game.result(state, a)
                    v2 = game.get_heuristic_score(player,new_state,a)
                        
                if DEPTH_LIMIT != 0:
                    v2, _ = min_value(game.result(state, a),DEPTH_LIMIT-1)
                    
                if v2 > v:
                    v, move = v2, a
                    
        return v, move

    def min_value(state, DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state):
             return game.utility(state, player), None
        v, move = +infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a,place):
                if DEPTH_LIMIT == 0:
                    new_state=game.result(state, a)
                    v2 = game.get_heuristic_score(player,new_state,a) 
                    
                if DEPTH_LIMIT != 0:
                    v2, _ = max_value(game.result(state, a),DEPTH_LIMIT-1)
                if v2 < v:
                    v, move = v2, a        
        return v, move
    
    if player=="O":
        v , move = max_value(state, DEPTH_LIMIT)
    else:
        v , move = min_value(state, DEPTH_LIMIT)
        
    return v, move

infinity = math.inf

    
def limited_alphabeta_search(game, state, DEPTH_LIMIT=0):
    """Search game to determine best action; use alpha-beta pruning. DEPTH_LIMIT is the depth limit of the search tree."""
    
    player = state.to_move
    
   
    
    def max_value(state, alpha, beta,DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state):
             return game.utility(state, player), None
        v, move = -infinity, None
        place = game.actions(state)
        
        for a in place:
            if game.has_neibour(a, place): 
                if DEPTH_LIMIT == 0: 
                    new_state = game.result(state, a)
                    v2 = game.get_heuristic_score(player,new_state,a) 
                
                if DEPTH_LIMIT != 0:
                    v2, _ = min_value(game.result(state, a), alpha, beta,DEPTH_LIMIT-1)
                    
                if v2 > v:
                    v, move = v2, a
                    alpha = max(alpha, v)
                    
                if alpha >= beta: # beta cut-off
                    return v, move
                
        return v, move

    def min_value(state, alpha, beta,DEPTH_LIMIT):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()   
        if game.is_terminal(state):
             return game.utility(state, player), None
        v, move = +infinity, None
        place = game.actions(state)
        for a in place:
            if game.has_neibour(a, place):
                if DEPTH_LIMIT == 0:
                    new_state=game.result(state, a)
                    v2 = game.get_heuristic_score(player,new_state,a)
                
                if DEPTH_LIMIT != 0:
                    v2, _ = max_value(game.result(state, a), alpha, beta,DEPTH_LIMIT-1)
                if v2 < v:
                    v, move = v2, a
                    beta = min(beta, v)
                if beta <= alpha: # alpha cut-off
                    return v, move
                
        return v, move
    
    if player=="O":
        v , move = max_value(state,-infinity, +infinity, DEPTH_LIMIT)
    else:
        v , move = min_value(state,-infinity, +infinity, DEPTH_LIMIT)
        
    return v, move

## A Simple Game: goBang ( Please complete the following code )


Moves are `(x, y)` pairs denoting squares, where `(0, 0)` is the top left, and `(16, 16)` is the bottom right (on a board of size `height=width=16`). Player need to have `k=5` same pieces in the same row or coloumn or diagonal to win the game. 

get_heuristic_score to divide the straight and slanting line into a string contain the square's coordinates, and send the string to board_status to calculate the score.

get_heuristic_score will be called by limited_alphabeta_search and limited_minimax_search


In [ ]:
 class goBang():
    """Play goBang on an `height` by `width` board, needing `k` in a row (colomn or diagnoal) to win.
    'X' plays first against 'O'."""

    def __init__(self, height=16, width=16, k=5):
        self.height = height
        self.width = width
        self.k = k 
        self.squares = {(x, y) for x in range(width) for y in range(height)}
        self.initial = Board(height=height, width=width, to_move='X', utility=0) #initially, the utility function is 0

        # visualize the game
        self.visualize = Visualize()
        self.visualize.draw_board()

    def actions(self, board):
        """Legal moves are any square not yet taken."""
        
        return self.squares - set(board)
    
    def result(self, board, square):
        """Place a marker for current player on square."""
        player = board.to_move
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        win = k_in_row(board, player, square, self.k)
        # 0 for not win, pow(1000,6) for win, -pow(1000,6) for lose
        board.utility = (0 if not win else + pow(1000,self.k+1) if player == 'X' else -pow(1000,self.k+1))      
        return board

    def utility(self, board, player):
        """Return the terminal state utility value corresponding to different players;
        pow(1000,6) for win, -pow(1000,6) for loss, 0 otherwise."""
        return board.utility if player == 'X' else -board.utility
    
    def is_terminal(self, board):
        """A board is a terminal state if it is won or there are no empty squares."""
        return board.utility != 0 or len(self.squares) == len(board)
    
    def has_neibour(self, square, place):  
        """Is this square has neibour?
        agent will not place a piece if it has no neibour pieces on the board
        this method can reduce the size of the problem by not examing pieces with no neibours"""
        
        x, y = square
        place_temp = place.copy() 

        # allow the first move to be placed as the neighbor of the center piece   
        if len(place_temp) == self.height*self.width:  # if the board is empty       
            place_temp.remove((round(self.height/2),round(self.width/2)))    
 
        for i in range(-1, 2):
            for j in range(-1, 2):
                 # identify the squares have neibour pieces
                if not ((x+i, y+j) in place_temp) and ((x+i, y+j) in self.squares):   
                    return True
        return False
    
    def board_status(self, board, player, array):
        
        score=0
        if player == "O":
            if len(array)>=6:
                lon = len(array)-6
                for i in range(0,lon+1):
                    arr = array[i:i+6]
                    
                    check=""
                    for j in range(0,6):  
                        
                        if j==0:
                            x0,y0=arr[j]
                            check+=board[x0,y0]
                            
                        if j==1:
                            x1,y1=arr[j]
                            check+=board[x1,y1]
                        if j==2:
                            x2,y2=arr[j]
                            check+=board[x2,y2]
                        if j==3:
                            x3,y3=arr[j]
                            check+=board[x3,y3]
                        if j==4:
                            x4,y4=arr[j]
                            check+=board[x4,y4] 
                        if j==5:
                            x5,y5=arr[j]
                            check+=board[x5,y5]
                    
                    if check == "XOOOOO" or check =="OOOOOX" or check ==".OOOOO" or check =="OOOOO.":
                        score+=1000000
                    elif check == ".OOOO.":
                        score+=300000    
                    elif  check=="OXXX.." or check=="OXXX.O":
                        score+=150000
                    elif check == "OXXXXO":
                        score += 400000
                    elif check== ".XOXX." or check == ".XXOX.":
                        score+=20000
                    elif check== ".O.OO." or check == ".OO.O.":
                        score+=10000
                    elif check == ".OOOOX" or check == "XOOOO.":
                        score+=6000
                    elif check == "..OOOX" or check==".O.OOX" or check==".OO.OX" or check == "XOOO.." or check=="XOO.O." or check=="XO.OO.":
                        score+=1000
                    elif check == "..OO..":
                        score+=600
                    elif check == "...OOX" or check=="..O.OX" or check==".O..OX" or check == "XOO..." or check=="XO..O." or check=="XO.O..":
                        score+=60
                    elif check == ".XXXX.":
                        score+=0
                    if check =="O.OXXX":
                        score-=400000
                    if check =="OXXX.O" or check == "O.XXXO":
                        score+=10000
                    if check=="XXXOO." or check == "OOXXX.":
                        score-=50000                    
                    if check == "O.XOXX":
                        score-=20000
                    if check =="XXXOXO" or check == "XXOXXO" or check=="XOXXXO":
                        score-=5000
                    if check=="OXXXOO" or check == "OOXXXO" or check == ".XXXOO" :
                        score-=4000000
                    if (check[0]=="O" or check[0]=="X") and (check[1]=="O" or check[1]=="X") and (check[2]=="O" or check[2]=="X"):
                        
                        break
                    

            if len(array)>=5:
                lon = len(array)-5
                for i in range(0,lon+1):
                    arr = array[i:i+5]
                    
                    check=""
                    for j in range(0,5):
                        
                        if j==0:
                            x0,y0=arr[j]
                            check+=board[x0,y0]
                            
                        if j==1:
                            x1,y1=arr[j]
                            check+=board[x1,y1]
                        if j==2:
                            x2,y2=arr[j]
                            check+=board[x2,y2]
                        if j==3:
                            x3,y3=arr[j]
                            check+=board[x3,y3]
                        if j==4:
                            x4,y4=arr[j]
                            check+=board[x4,y4] 
                    
                    if check == "OOOOO":
                        score+=1000000                    
                    elif check == "XXXXX":
                        score+=0
                    elif check=="XOXXX" or check=="XXOXX" or check =="XXXOX":
                        score+=300000
                    elif check== ".XXXO" or check=="OXXX.":
                        score+=150000
                    elif check == "XOXXX" or check == "XXOXX" or check =="XXXOX":
                        score+=12000
                    elif check== ".OOO.":
                        score+=10000
                    elif check == "O.OOO" or check == "OO.OO" or check =="OOO.O":
                        score+=6000
                    elif check == "O..OO" or check=="O.O.O" or check =="OO..O":
                        score+=1000
                    elif check == ".O.O.":
                        score+=600
                    elif check == "O...O":
                        score+=60
                        
                    if check=="XXXOO" or check == "OOXXX" or check == ".XXX." or check == "XX.XX":
                        score-=4000000
                    
                    
                    if (check[1]=="O" or check[1]=="X") and (check[1]=="O" or check[1]=="X"):
                        
                        break
            
        

        if player == "X":
            if len(array)>=6:
                lon = len(array)-6
                for i in range(0,lon+1):
                    arr = array[i:i+6]
                    
                    check=""
                    for j in range(0,6):  
                        
                        if j==0:
                            x0,y0=arr[j]
                            check+=board[x0,y0]
                            
                        if j==1:
                            x1,y1=arr[j]
                            check+=board[x1,y1]
                        if j==2:
                            x2,y2=arr[j]
                            check+=board[x2,y2]
                        if j==3:
                            x3,y3=arr[j]
                            check+=board[x3,y3]
                        if j==4:
                            x4,y4=arr[j]
                            check+=board[x4,y4] 
                        if j==5:
                            x5,y5=arr[j]
                            check+=board[x5,y5]
                    
                    if check == "OXXXXX" or check =="XXXXXO" or check ==".XXXXX" or check =="XXXXX.":
                        score-=1000000
                    elif check == ".XXXX.":
                        score-=300000    
                    elif  check=="XOOO.." or check=="XOOO.X":
                        score-=150000
                    elif check == "XOOOOX":
                        score -= 400000
                    elif check== ".OXOO." or check == ".OOXO.":
                        score-=20000
                    elif check== ".X.XX." or check == ".XX.X.":
                        score-=10000
                    elif check == ".XXXXO" or check == "OXXXX.":
                        score-=6000
                    elif check == "..XXXO" or check==".X.XXO" or check==".XX.XO" or check == "OXXX.." or check=="OXX.X." or check=="OX.XX.":
                        score-=1000
                    elif check == "..XX..":
                        score-=600
                    elif check == "...XXO" or check=="..X.XO" or check==".X..XO" or check == "OXX..." or check=="OX..X." or check=="OX.X..":
                        score-=60
                    elif check == ".OOOO.":
                        score-=0
                    if check =="X.XOOO":
                        score+=400000
                    if check =="XOOO.X" or check == "X.OOOX":
                        score-=10000
                    if check=="OOOXX." or check == "XXOOO.":
                        score+=50000                    
                    if check == "X.OXOO":
                        score+=20000
                    if check =="OOOXOX" or check == "OOXOOX" or check=="OXOOOX":
                        score+=5000
                    if check=="XOOOXX" or check == "XXOOOX" or check == ".OOOXX":
                        score+=4000000
                    if (check[0]=="X" or check[0]=="O") and (check[1]=="X" or check[1]=="O") and (check[2]=="X" or check[2]=="O"):
                        break
                    

            if len(array)>=5:
                lon = len(array)-5
                for i in range(0,lon+1):
                    arr = array[i:i+5]
                    
                    check=""
                    for j in range(0,5):
                        
                        if j==0:
                            x0,y0=arr[j]
                            check+=board[x0,y0]
                            
                        if j==1:
                            x1,y1=arr[j]
                            check+=board[x1,y1]
                        if j==2:
                            x2,y2=arr[j]
                            check+=board[x2,y2]
                        if j==3:
                            x3,y3=arr[j]
                            check+=board[x3,y3]
                        if j==4:
                            x4,y4=arr[j]
                            check+=board[x4,y4] 
                    
                    if check == "XXXXX":
                        score-=1000000                    
                    elif check == "OOOOO":
                        score-=0
                    elif check=="OXOOO" or check=="OOXOO" or check =="OOOXO":
                        score-=300000
                    elif check== ".OOOX" or check=="XOOO.":
                        score-=150000
                    elif check == "OXOOO" or check == "OOXOO" or check =="OOOXO":
                        score-=12000
                    elif check== ".XXX.":
                        score-=10000
                    elif check == "X.XXX" or check == "XX.XX" or check =="XXX.X":
                        score-=6000
                    elif check == "X..XX" or check=="X.X.X" or check =="XX..X":
                        score-=1000
                    elif check == ".X.X.":
                        score-=600
                    elif check == "X...X":
                        score-=60
                        
                    if check=="OOOXX" or check == "XXOOO" or check == ".OOO." or check == "OO.OO":
                        score+=4000000
                    
                    
                    if (check[0]=="X" or check[0]=="O") and (check[1]=="X" or check[1]=="O"):
                        
                        break
        
        return score
        
        
    def get_heuristic_score(self, player ,board, square):
        
        x,y = square
        
        for xs1 in range(5,-1,-1):
            if x-xs1 >= 0:
                x1=x-xs1
                
                break
                
        for ys1 in range(5,-1,-1):
            if y-ys1 >= 0:
                y1=y-ys1
                
                break
        x2=x1
        
        if(x-x1>y-y1):
            x1=x-(y-y1)
        elif(x-x1<y-y1):
            y1=y-(x-x1)
        
        
        for ys2 in range(5,-1,-1):
            if y+ys2 <= 15:
                y2=y+ys2
                break
            
        if(x-x2>y2-y):
            x2=x-(y2-y)
        elif(x-x2<y2-y):
            y2=y+(x-x2)
        
        
        arr1 = [[x1,y1]]
        arr2 = [[x1,y]]
        arr3 = [[x,y1]]
        arr4 = [[x2,y2]]
        for ran in range(1,11):
            x1+=1
            y1+=1
            if(x1<=15 and y1<=15):
                arr1.append([x1,y1])
                arr2.append([x1,y])
                arr3.append([x,y1])
        for ran in range(1,11):
            x2+=1
            y2-=1
            if(x2<=15 and y2>=0):
                arr4.append([x2,y2])
        if player == "X":
            print(arr1)
            print(arr2)
            print(arr3)
            print(arr4)
            
        score=0;
        
        score=self.board_status(board, player ,arr1)+self.board_status(board, player ,arr2)+self.board_status(board, player ,arr3)+self.board_status(board, player ,arr4)
  
        return score

    def board_score(self, board, square):
        """Utilize the heuristic method to estimate the utility of the game board
        that applying potential move specified by `squre`. 
        The new board's utility estimation takes both players into consideration. """
        
        player = board.to_move
        
        #update the board by performing the move specified by squre
        board = board.new({square: player}, to_move=('O' if player == 'X' else 'X'))
        #calculate the board estimated utility taking both 
        adjust_weight = 0.5  # you can try a different adjust_weight to see what will happen
        board_score = self.get_heuristic_score(board, player) - adjust_weight * self.get_heuristic_score(board, ('O' if player == 'X' else 'X'))
        return board_score
    

    def display(self, board): print(board)


def k_in_row(board, player, square, k):
    """True if player has k pieces in a line through square."""
    def in_row(x, y, dx, dy): return 0 if board[x, y] != player else 1 + in_row(x + dx, y + dy, dx, dy)
    return any(in_row(*square, dx, dy) + in_row(*square, -dx, -dy) - 1 >= k
               for (dx, dy) in ((0, 1), (1, 0), (1, 1), (1, -1)))


## Implementation of different players playing the game

Implement how players interact with the game, and how strategies are assigned to different players. I'll represent a player as a `callable` that will be passed two arguments: `(game, state)` and will return a `move`.
The function `player` creates a player out of a search algorithm, but you can create your own players as functions, as is done with `random_player` below. You can leave the following code section unchanged.XD

In [ ]:
def play_game(game, strategies: dict, verbose=False):
    """Play a turn-taking game. `strategies` is a {player_name: function} dict,
    where function(state, game) is used to get the player's move."""
    state = game.initial
    visualize = game.visualize
    
    while not game.is_terminal(state): 
        player = state.to_move
        move = strategies[player](game, state)
        state = game.result(state, move)
        k_in_row(state,player,move,0)
        visualize.draw_piece(move, visualize.BLACK if player == 'X' else visualize.WHITE)
        print('Player', player, ':', move)
       
        
        if verbose: 
            print(state)
    print('Game over: utility', game.utility(state, 'X'), 'for X')
    
    # exit or restart the GUI
    again = visualize.draw_end_screen(1 if game.utility(state, 'X') > 0 else 2 if game.utility(state, 'X') < 0 else 0)
    if again == 0:
        main()
    else:
        pygame.quit()
        sys.exit()

    return state

In [ ]:
def random_player(game, state): 
    if state.to_move == 'O':
        color = "white"
    else:
        color = "black"
    (x,y)=random.choice(list(game.actions(state)))
    game.visualize.draw_piece((x,y),color)
    return (x,y)

def human_player(game, state):
    """Make a move by reading input."""
    print(state)
    print('Legal moves are', game.actions(state))
    move = None
    while move not in game.actions(state):
        move = eval(input('Your move? '))
    # vis.draw_piece(move,"black")
    print(move)
    return move

def human_mouse_player(game, state):
    """Make a move by reading input."""
    move = None
    while move not in game.actions(state):
        move= game.visualize.get_input()
    return move

def player(search_algorithm):
    """A game player who uses the specified search algorithm"""
    return lambda game, state: search_algorithm(game, state)[1]

## Playing a Game

The main function that excuting the goBang game.

In [ ]:
def main():
    #play_game(goBang(), dict(X=human_mouse_player, O=human_mouse_player), verbose=False).utility
    #play_game(goBang(), dict(X=player(limited_alphabeta_search), O=human_mouse_player), verbose=False).utility
    play_game(goBang(), dict(X=human_mouse_player, O=player(limited_alphabeta_search)), verbose=False).utility
    #play_game(goBang(), dict(X=player(limited_alphabeta_search), O=player(limited_alphabeta_search)), verbose=False).utility
    #play_game(goBang(), dict(X=player(limited_alphabeta_search), O=player(limited_minimax_search)), verbose=False).utility
    #play_game(goBang(), dict(X=human_mouse_player, O=player(limited_minimax_search)), verbose=False).utility
if __name__ == '__main__':
    main()